# Basic liquid handling

In this notebook, you will learn how to use PyHamilton to move water from one range of wells to another.

**Note: before running this notebook, you should have**:

- Installed PyHamilton and the USB driver as described in [the installation guide](installation).
- Connected the Hamilton to your computer using the USB cable.

Video of what this code does:

<iframe width="640" height="360" src="https://www.youtube.com/embed/NN6ltrRj3bU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Setting up a connection with the robot

Start by importing the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class, which will serve as a front end for all liquid handling operations.

Backends serve as communicators between between `LiquidHandler`s and the actual hardware. Since we are using a Hamilton STAR, we also import the {class}`~pylabrobot.liquid_handling.backends.STAR` backend.

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
!python.exe -m pip install --upgrade pip

In [2]:
!pip install --upgrade opentrons

In [3]:
!pip install pylabrobot[opentrons]

In [10]:
### RUN THIS FIRST TIME YOU USE PLR!
# import sys
# sys.path.append(r"C:\Users\vikmol\Documents\GitHub\pylabrobot_DALSA")


In [1]:
from pylabrobot.liquid_handling import LiquidHandler
# from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.liquid_handling.backends import OpentronsBackend

# from pylabrobot.liquid_handling.backends.opentrons_backend import OpentronsBackend

In addition, import the {class}`~pylabrobot.resources.hamilton.STARLetDeck`, which represents the deck of the Hamilton STAR.

In [2]:
# from pylabrobot.resources.hamilton import STARLetDeck
from pylabrobot.resources.opentrons.deck import OTDeck

In [10]:
## OPS:: EXTRA PART ADD SEARCH FOR THE CORRECT TYPE OF USB CABLE BY VIEWING IP ADRESSES

Create a new liquid handler using `STAR` as its backend.

In [3]:
# backend = STAR()
# lh = LiquidHandler(backend=backend, deck=STARLetDeck())
# lh = LiquidHandler(backend=OpentronsBackend(host="10.199.253.164"), deck=OTDeck())
lh = LiquidHandler(backend=OpentronsBackend(host="10.199.253.164"), deck=OTDeck(no_trash=True)) # This seems to work
# web: 10.199.253.164 (OT032)
# web: 10.199.253.141 (OT006)
# web: 10.199.253.233 (OT028)
# lan: 169.254.131.46
# backend = OpentronsBackend(host="0.0.0.0", port=5001)
# lh = LiquidHandler(backend=backend, deck=OTDeck())

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [4]:
await lh.setup()

ConnectTimeout: HTTPConnectionPool(host='10.199.253.164', port=31950): Max retries exceeded with url: /runs (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001A27EDCD6C0>, 'Connection to 10.199.253.164 timed out. (connect timeout=None)'))

## Creating the deck layout

Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyHamilton package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pylabrobot.resources.ml_star.tip_racks.HTF_L` tips

In [7]:
from pylabrobot.resources.opentrons import (
  opentrons_96_filtertiprack_20ul
)
from pylabrobot.resources.corning_costar import (
  Cos_96_EZWash,
  Cos_96_DW_1mL
)


Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [8]:
tips1 = opentrons_96_filtertiprack_20ul(name="tip_rack1")
tips2 = opentrons_96_filtertiprack_20ul(name="tip_rack2")
plate1 = Cos_96_EZWash(name="plate_01")
plate2 = Cos_96_DW_1mL(name="plate_02")

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [9]:
lh.deck.assign_child_at_slot(tips1, slot=3)
lh.deck.assign_child_at_slot(tips2, slot=5)
lh.deck.assign_child_at_slot(plate1, slot=11)
lh.deck.assign_child_at_slot(plate2, slot=10)

Repeat this for the plates.

In [10]:
# summary function that displayes deck
print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



In [11]:
# get slot position of resource
lh.deck.get_slot(tips1)

3

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [12]:
lh.summary()


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



## Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [32]:
from pylabrobot.resources.carrier import Coordinate


tiprack = lh.get_resource("tip_rack1")
## OPS OPS CHANGE TO SINGLE CHANNEL OG MULTI CHANNEL FOR THE OT ROBOT
await lh.pick_up_tips(tiprack["A1"], offsets=Coordinate(0,0,-7))


## Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [33]:
## Calibrations are compleatly off!!

plate = lh.get_resource("plate_01")
await lh.aspirate(plate["A1"], vols=[15])

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [34]:
## Calibrations are compleatly off!!

await lh.dispense(plate["F1"], vols=[15])

Let's move the liquid back to the original wells.

In [15]:
# await lh.aspirate(plate["D1:F1"], vols=[100.0, 50.0, 200.0])
# await lh.dispense(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0ASid0008at0&tm1 1 1 0&xp04330 04330 04330 00000&yp1190 1100 1010 0000&th2450te2450lp1931 1931 1931&ch000 000 000&zl1881 1881 1881&po0100 0100 0100&zu0032 0032 0032&zr06180 06180 06180&zx1831 1831 1831&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&av01072 00551 02110&as1000 1000 1000&ta000 000 000&ba0000 0000 0000&oa000 000 000&lm0 0 0&ll1 1 1&lv1 1 1&zo000 000 000&ld00 00 00&de0020 0020 0020&wt10 10 10&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms1000 1000 1000&mh0000 0000 0000&gi000 000 000&gj0gk0lk0 0 0&ik0000 0000 0000&sd0500 0500 0500&se0500 0500 0500&sz0300 0300 0300&io0000 0000 0000&il00000 00000 00000&in0000 0000 0000&
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0ASid0008er00/00
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0DSid0009dm2 2 2&tm1 1 1 0&xp04330 04330 04330 00000&yp1460 1370 1280 0000&zx1871 1871 1871&lp2321 2321 2321&zl1881 1881 1881&po0100 01

## Discarding tips

Finally, you can discard tips by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [35]:
await lh.drop_tips(tiprack["A1"])

# TEST Throughing tip into trash

In [8]:
from pylabrobot.liquid_handling import LiquidHandler

from pylabrobot.liquid_handling.backends import OpentronsBackend

from pylabrobot.resources.opentrons.deck import OTDeck

# lh = LiquidHandler(backend=OpentronsBackend(host="169.254.7.160"), deck=OTDeck(no_trash=False)) # This seems to work
lh = LiquidHandler(backend=OpentronsBackend(host="169.254.178.135"), deck=OTDeck(no_trash=True)) # This seems to work

await lh.setup()

from pylabrobot.resources.opentrons import (
  opentrons_96_tiprack_20ul,
  nest_96_wellplate_200ul_flat,
  opentrons_96_tiprack_300ul
)
from pylabrobot.resources.corning_costar import (
  Cos_96_EZWash,
  Cos_96_DW_1mL,
)

# tips1 = opentrons_96_tiprack_20ul(name="tip_rack1")
tips1 = opentrons_96_tiprack_300ul(name='tip_rack1')
tips2 = opentrons_96_tiprack_300ul(name="tip_rack2")
plate1 = nest_96_wellplate_200ul_flat(name="plate_01")#Cos_96_EZWash(name="plate_01")
plate2 = nest_96_wellplate_200ul_flat(name="plate_02")#Cos_96_DW_1mL(name="plate_02")

lh.deck.assign_child_at_slot(tips1, slot=3)
lh.deck.assign_child_at_slot(tips2, slot=5)
lh.deck.assign_child_at_slot(plate1, slot=11)
lh.deck.assign_child_at_slot(plate2, slot=10)

print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



In [6]:
tiprack = lh.get_resource("tip_rack1")
await lh.pick_up_tips(tiprack["A1"])


In [7]:
await lh.drop_tips(tiprack["A1"])

In [4]:
import ot_api

ot_api.requestor.get(f'/runs/{ot_api.run_id}/loaded_labware_definitions')


{'data': [{'schemaVersion': 2,
   'version': 1,
   'namespace': 'pylabrobot',
   'metadata': {'displayName': 'tip_rack1',
    'displayCategory': 'tipRack',
    'displayVolumeUnits': 'µL'},
   'brand': {'brand': 'unknown'},
   'parameters': {'format': '96Standard',
    'isTiprack': True,
    'tipLength': 39.2,
    'tipOverlap': 3.29,
    'loadName': 'tip_rack1',
    'isMagneticModuleCompatible': False},
   'ordering': [['tip_rack1_A1',
     'tip_rack1_B1',
     'tip_rack1_C1',
     'tip_rack1_D1',
     'tip_rack1_E1',
     'tip_rack1_F1',
     'tip_rack1_G1',
     'tip_rack1_H1'],
    ['tip_rack1_A2',
     'tip_rack1_B2',
     'tip_rack1_C2',
     'tip_rack1_D2',
     'tip_rack1_E2',
     'tip_rack1_F2',
     'tip_rack1_G2',
     'tip_rack1_H2'],
    ['tip_rack1_A3',
     'tip_rack1_B3',
     'tip_rack1_C3',
     'tip_rack1_D3',
     'tip_rack1_E3',
     'tip_rack1_F3',
     'tip_rack1_G3',
     'tip_rack1_H3'],
    ['tip_rack1_A4',
     'tip_rack1_B4',
     'tip_rack1_C4',
     'tip_ra

In [3]:
from pylabrobot.resources.trash import Trash
# from pylabrobot.resources.opentrons.deck import OTDeck

In [4]:
# trash = lh.deck.get_trash_area()
# # n must match with the number of pipets
# n=1
# OTDeck._assign_trash()
lh.deck._assign_trash()

# lh.deck._assign_trash()


# trash_container.assign_child_resource(actual_trash, location=Coordinate(x=82.84, y=53.56, z=5))
# lh.deck.assign_child_at_slot(lh.deck._assign_trash(), 12)


Exception in thread Thread-10 (callback):
Traceback (most recent call last):
  File "C:\Users\vikmol\AppData\Local\Programs\Python\Python310\Lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\vikmol\AppData\Local\Programs\Python\Python310\Lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\liquid_handler.py", line 163, in callback
    loop.run_until_complete(func(*args, **kwargs))
  File "C:\Users\vikmol\AppData\Local\Programs\Python\Python310\Lib\asyncio\base_events.py", line 649, in run_until_complete
    return future.result()
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\opentrons_backend.py", line 219, in assigned_resource_callback
    ot_api.labware.add(
  File "c:\Users\vikmol\OneDrive\Dokumenter\GitHub\pylabrobot_DALSA\venv\lib\site-packages\ot_api\decorators.py", lin

In [6]:
# Trash.assign_child_resource(actual_trash, location=Coordinate(x=82.84, y=53.56, z=5))


In [5]:
trash = lh.deck.get_trash_area()

In [12]:
# lh.deck.assign_child_at_slot(trash, 11)
# trash_container.assign_child_resource(actual_trash, location=Coordinate(x=82.84, y=53.56, z=5))

# lh.deck.assign_child_at_slot(trash, slot=12)

ValueError: Spot 12 is already occupied

In [6]:
print(lh.deck.summary())



Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tip_rack2   |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tip_rack1   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



In [7]:
print(lh.backend.defined_labware)

{'tip_rack1': 'tip_rack1', 'tip_rack2': 'tip_rack2', 'plate_01': 'plate_01', 'plate_02': 'plate_02'}


In [8]:
await lh.drop_tips(tip_spots=[trash]*1)

# await lh.drop_tips(

KeyError: 'trash_container'

In [15]:
await lh.stop()

In [ ]:
!python -c "import opentrons.protocol_engine.errors as ot_errors; print(getattr(ot_errors, 'AreaNotInDeckConfigurationError'))"

In [22]:
!pip freeze
#| grep opentrons

accessible-pygments==0.0.4
aionotify==0.2.0
alabaster==0.7.13
anyio==3.3.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asgiref==3.7.2
astroid==3.0.2
asttokens==2.4.1
async-lru==2.0.4
attrs==23.1.0
Babel==2.14.0
beautifulsoup4==4.12.2
bleach==6.1.0
blinker==1.7.0
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
clr-loader==0.2.6
colorama==0.4.6
comm==0.2.0
debugpy==1.8.0
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.7
docutils==0.19
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.0
Flask==3.0.0
fqdn==1.5.1
greenlet==3.0.2
idna==3.6
imagesize==1.4.1
importlib-metadata==7.0.0
iniconfig==2.0.0
ipykernel==6.27.1
ipython==8.18.1
ipywidgets==8.1.1
isoduration==20.11.0
isort==5.13.2
itsdangerous==2.1.2
jedi==0.19.1
Jinja2==3.1.2
json5==0.9.14
jsonpointer==2.4
jsonschema==3.0.2
jsonschema-specifications==2023.12.1
jupyter==1.0.0
jupyter-cache==1.0.0
jupyter-console==6.6.3
jupyter-events==0.9.0
jupyter-lsp==2.2.2
jupyter_client==8.6.0
jupyter_c

# Test robot calibration


In [4]:
tiprack = lh.get_resource("tip_rack1")
await lh.pick_up_tips(tiprack["A1"])
# await lh.drop_tips(tiprack["A1"])

In [5]:
await lh.drop_tips(tiprack["A1"])